In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples
from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.splitter import SubsampleSplitter

from reversible2.view_as import ViewAs

from reversible2.affine import AdditiveBlock
from reversible2.plot import display_text, display_close
from reversible2.bhno import load_file, create_inputs
th.backends.cudnn.benchmark = True

In [ ]:
sensor_names = ['Fz', 
                'FC3','FC1','FCz','FC2','FC4',
                'C5','C3','C1','Cz','C2','C4','C6',
                'CP3','CP1','CPz','CP2','CP4',
                'P1','Pz','P2',
                'POz']
orig_train_cnt = load_file('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat')
train_cnt = orig_train_cnt.reorder_channels(sensor_names)

train_inputs = create_inputs(train_cnt, final_hz=256, half_before=True,
                            start_ms=500, stop_ms=1500)
n_split = len(train_inputs[0]) - 40
test_inputs = [t[-40:] for t in train_inputs]
train_inputs = [t[:-40] for t in train_inputs]

In [ ]:
cuda = True
if cuda:
    train_inputs = [i.cuda() for i in train_inputs]
    test_inputs = [i.cuda() for i in test_inputs]

In [ ]:
feature_model = th.load('/data/schirrmr/schirrmr/reversible/models/notebooks/21ChansOT/feature_model.pkl')

class_dist = th.load('/data/schirrmr/schirrmr/reversible/models/notebooks/21ChansOT/class_dist.pkl')

In [ ]:
class_dist.i_class_inds = [0,1]

In [ ]:
from reversible2.plot import plot_head_signals_tight
inverted_per_class = []
for i_class in range(2):
    samples = class_dist.get_mean_std(i_class)[0].unsqueeze(0)
    inverted = feature_model.invert(samples)
    inverted_per_class.append(var_to_np(inverted)[0].squeeze())
fig = plot_head_signals_tight(np.stack(inverted_per_class, axis=-1), sensor_names=sensor_names,
                         figsize=(20,12));
ax = plt.gca()

fig.legend(ax.get_lines()[:2], ["Right Hand", "Resting State"], fontsize=24,loc='upper right',
          bbox_to_anchor=(0.79,0.81))
#plt.savefig('/data/schirrmr/schirrmr/ohbm-rom/bigsignal.png', dpi=300 ,transparent=True,
#           bbox_inches='tight', pad_inches=0)

In [ ]:
plt.plot(var_to_np(class_dist.get_mean_std(1)[1]))

In [ ]:
from reversible2.plot import plot_head_signals_tight

from reversible2.constantmemory import clear_ctx_dicts
inverted_per_class = []
n_examples = 21
for i_class in range(2):
    samples = class_dist.get_mean_std(i_class)[0].unsqueeze(0)
    samples = samples.repeat(n_examples,1)

In [ ]:
seaborn.set_context('notebook', font_scale=1.75)

mean, std = var_to_np(class_dist.get_mean_std(i_class)[0]), var_to_np(class_dist.get_mean_std(i_class)[1])
np.argsort(std)[::-1][:10]

for i_i_std in range(5):
    samples = class_dist.get_mean_std(i_class)[0].unsqueeze(0)
    samples = samples.repeat(n_examples,1)
    i_std = np.argsort(var_to_np(class_dist.get_mean_std(1)[1]))[::-1][i_i_std]
    print("std", i_std, std[i_std])

    interps = np.linspace(mean[i_std] - 1.5 *std[i_std], mean[i_std] + 1.5 *std[i_std], n_examples)

    samples.data[:,i_std] = np_to_var(interps, device='cuda', dtype=np.float32)
    inverted = feature_model.invert(samples)
    clear_ctx_dicts(feature_model)
    from matplotlib import rcParams, cycler
    cmap = plt.cm.coolwarm
    with plt.rc_context(rc=({'axes.prop_cycle': cycler(color=cmap(np.linspace(0, 1, n_examples)))})):
        
        # Setting up a colormap that's a simple transtion
        mymap = matplotlib.colors.LinearSegmentedColormap.from_list('mycolors',['blue','red'])

        # Using contourf to provide my colorbar info, then clearing the figure
        Z = [[0,0],[0,0]]
        levels = range(0, n_examples)
        CS3 = plt.contourf(Z, levels, cmap=mymap)
        plt.clf()
        
        fig = plot_head_signals_tight(var_to_np(inverted).squeeze().transpose(1,2,0), sensor_names=sensor_names,
                                 figsize=(20,12));
        cbar = plt.colorbar(CS3, anchor=(1,2), panchor=(1,2),ax=fig.get_axes()[0])
        cbar.set_ticklabels(["min",] + ([""] * (len(cbar.get_ticks()) - 2)) +["max"])
        cbar.set_label("Latent value", fontsize=16)
        plt.savefig('/data/schirrmr/schirrmr/ohbm-rom/visualization{:d}.png'.format(i_i_std),
                    dpi=300 ,transparent=False,
                   bbox_inches='tight', pad_inches=0)
        
        display_close(fig)
seaborn.set_context('poster')

In [ ]:
from reversible2.plot import plot_head_signals_tight
inverted_per_class = []
for i_class in range(2):
    with th.no_grad():
        samples = class_dist.get_samples(i_class, len(train_inputs[i_class]) * 4)
        inverted = feature_model.invert(samples)
    clear_ctx_dicts(feature_model)

In [ ]:
import ot
def transport_mat_for_diffs(samples_a, samples_b):
    diffs = samples_a.unsqueeze(1) - samples_b.unsqueeze(0)
    diffs = th.norm(diffs, dim=2, p=2)
    transport_mat = ot.emd([], [], var_to_np(diffs))
    # sometimes weird low values, try to prevent them
    # 0.5 could be 1.0, just set ot 0.5 to make more sure nothing
    # removed accidentally
    transport_mat = transport_mat * (transport_mat >= (0.5 / (diffs.numel())))
    return transport_mat

In [ ]:
train_inputs[0].shape

In [ ]:
inverted.shape

In [ ]:
t_mat = transport_mat_for_diffs(train_inputs[i_class].view(len(train_inputs[i_class]),-1),
                        inverted.view(len(inverted), -1))

In [ ]:
t_mat.shape

In [ ]:
import matplotlib.style as style
style.use('seaborn-poster')
seaborn.set_context('poster')

seaborn.set_palette("colorblind", )



In [ ]:
for i_row in range(5, t_mat.shape[0]):
    matches = np.flatnonzero(t_mat[i_row])
    signal = var_to_np(train_inputs[i_class][i_row]).squeeze()
    inv_matches = var_to_np(inverted[matches]).squeeze()

    if i_row > 7:
        break

    diffs = np.linalg.norm(signal[None] - inv_matches, ord=2, axis=2).squeeze()

    i_t, i_c = np.argmin(diffs) // 22, np.argmin(diffs) % 22

    with seaborn.axes_style("white"):
        fig = plt.figure(figsize=(8,2))
        plt.plot(signal[i_c], color=seaborn.color_palette()[2])
        plt.plot(inv_matches[i_t][i_c], color=seaborn.color_palette()[3])
        plt.axis('off')
        plt.legend(["Real signal", "Matched generated signal"], bbox_to_anchor=(1,1,0,0))
        plt.savefig('/data/schirrmr/schirrmr/ohbm-rom/match_legend{:d}.png'.format(i_row), dpi=300 ,transparent=True,
           bbox_inches='tight', pad_inches=0)
        display_close(fig)
        

In [ ]:
for i_row in range(5, t_mat.shape[0]):
    matches = np.flatnonzero(t_mat[i_row])
    signal = var_to_np(train_inputs[i_class][i_row]).squeeze()
    inv_matches = var_to_np(inverted[matches]).squeeze()
    
    if i_row > 7:
        break

    diffs = np.linalg.norm(signal[None] - inv_matches, ord=2, axis=2).squeeze()

    i_t, i_c = np.argmin(diffs) // 22, np.argmin(diffs) % 22

    with seaborn.axes_style("white"):
        fig = plt.figure(figsize=(8,2))
        plt.plot(signal[i_c], color=seaborn.color_palette()[2])
        plt.plot(inv_matches[i_t][i_c], color=seaborn.color_palette()[3])
        plt.axis('off')
        plt.savefig('/data/schirrmr/schirrmr/ohbm-rom/match_{:d}.png'.format(i_row), dpi=300 ,transparent=True,
           bbox_inches='tight', pad_inches=0)
        display_close(fig)

In [ ]:
from reversible2.plot import plot_head_signals_tight
inverted_per_class = []
for i_class in range(2):
    with th.no_grad():
        samples = class_dist.get_samples(i_class, len(train_inputs[i_class]) * 4)
        inverted = feature_model.invert(samples)
        inverted = var_to_np(inverted).squeeze()
        signals = var_to_np(train_inputs[i_class]).squeeze()
    clear_ctx_dicts(feature_model)
    

In [ ]:
fig = plt.figure(figsize=(8,2))
plt.plot(np.fft.rfftfreq(256,d=1.0/256.0), np.mean(np.abs(np.fft.rfft(signals)), axis=(0,1)),
         color=seaborn.color_palette()[2])
plt.plot(np.fft.rfftfreq(256,d=1.0/256.0), np.mean(np.abs(np.fft.rfft(inverted)), axis=(0,1)),
         color=seaborn.color_palette()[3])
plt.xlabel("Frequency [Hz]")
plt.ylabel("Amplitude")
plt.legend(["Real Resting State", "Generated"], fontsize=18,loc='upper right',
          bbox_to_anchor=(1,1))
plt.savefig('/data/schirrmr/schirrmr/ohbm-rom/overall_spectrum.png', dpi=300 ,transparent=False,
   bbox_inches='tight', pad_inches=0)

In [ ]:
with plt.rc_context(rc=({'axes.prop_cycle': cycler(color=seaborn.color_palette()[2:4])})):
    fig = plot_head_signals_tight(np.stack([np.mean(np.abs(np.fft.rfft(signals)), axis=0),
             np.mean(np.abs(np.fft.rfft(inverted)), axis=0)],axis=-1),
                                  sensor_names=sensor_names,
                                     figsize=(20,12));
    plt.ylim(0,55)
    ax = plt.gca()

    fig.legend(ax.get_lines()[:2], ["Real Resting State", "Generated"], fontsize=24,loc='upper right',
              bbox_to_anchor=(0.77,0.79))
    
    plt.savefig('/data/schirrmr/schirrmr/ohbm-rom/spectrum_by_chan.png', dpi=300 ,transparent=False,
       bbox_inches='tight', pad_inches=0)
    
    display_close(fig)

In [ ]:
state_dict = th.load('/data/schirrmr/schirrmr/models/eegconvnet-public-data-simple-no-multiplication/pretrained/deep/4.pkl')

In [ ]:
train_set.X.shape

In [ ]:
from braindecode.datautil.signal_target import SignalAndTarget
def inputs_to_set(inputs):
    X = np.concatenate((var_to_np(inputs[0]),
                      var_to_np(inputs[1])), 
                          ).astype(np.float32).squeeze(-1)
    y = np.concatenate((np.zeros(len(inputs[0]), dtype=np.int64),
                       np.ones(len(inputs[1]), dtype=np.int64)))
    return SignalAndTarget(X, y)

In [ ]:
train_set = inputs_to_set(train_inputs)
test_set = inputs_to_set(test_inputs)

In [ ]:
train_set.X.shape[2]

In [ ]:
from braindecode.models.deep4 import Deep4Net

from torch import nn
from braindecode.torch_ext.util import set_random_seeds

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = True
set_random_seeds(seed=20170629, cuda=cuda)

n_chans = train_set.X.shape[1]
n_classes = 2
input_time_length = train_set.X.shape[2]
model = Deep4Net(n_chans, n_classes,
                 input_time_length=input_time_length,
                 pool_time_length=2,
                 pool_time_stride=2,
                 final_conv_length='auto')

In [ ]:
from braindecode.torch_ext.optimizers import AdamW
import torch.nn.functional as F
optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
#optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)

model.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1,)

In [ ]:
model.fit(train_set.X, train_set.y, epochs=30, batch_size=64, scheduler='cosine',
         validation_data=(test_set.X, test_set.y),)

In [ ]:
from braindecode.torch_ext.optimizers import AdamW
import torch.nn.functional as F
optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model

model.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1,)

In [ ]:
from reversible2.plot import plot_head_signals_tight
set_random_seeds(201905231, cuda)
with th.no_grad():
    inverted_per_class = []
    for i_class in range(2):
        samples = class_dist.get_samples(i_class, 184)
        inverted = feature_model.invert(samples)
        inverted_per_class.append(inverted.detach())

from reversible2.constantmemory import clear_ctx_dicts
clear_ctx_dicts(feature_model)

In [ ]:
sample_set = inputs_to_set(inverted_per_class)

In [ ]:
model.evaluate(sample_set.X, sample_set.y)

In [ ]:
from braindecode.models.deep4 import Deep4Net

from torch import nn
from braindecode.torch_ext.util import set_random_seeds

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = True
set_random_seeds(seed=20170629, cuda=cuda)

n_chans = sample_set.X.shape[1]
n_classes = 2
input_time_length = sample_set.X.shape[2]
sample_model = Deep4Net(n_chans, n_classes,
                 input_time_length=input_time_length,
                 pool_time_length=2,
                 pool_time_stride=2,
                 final_conv_length='auto')
from braindecode.torch_ext.optimizers import AdamW
import torch.nn.functional as F
optimizer = AdamW(sample_model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
#optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)

sample_model.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1,)

In [ ]:
sample_model.fit(sample_set.X, sample_set.y, epochs=30, batch_size=64, scheduler='cosine',
         validation_data=(test_set.X, test_set.y),)

In [ ]:
from reversible2.plot import plot_head_signals_tight
set_random_seeds(201905231, cuda)
with th.no_grad():
    inverted_per_class = []
    for i_class in range(2):
        samples = class_dist.get_samples(i_class, 184*3)
        inverted = feature_model.invert(samples)
        inverted_per_class.append(inverted.detach())

from reversible2.constantmemory import clear_ctx_dicts
clear_ctx_dicts(feature_model)

In [ ]:
sample_set = inputs_to_set(inverted_per_class)

In [ ]:
from braindecode.models.deep4 import Deep4Net

from torch import nn
from braindecode.torch_ext.util import set_random_seeds

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = True
set_random_seeds(seed=20170629, cuda=cuda)

n_chans = sample_set.X.shape[1]
n_classes = 2
input_time_length = sample_set.X.shape[2]
sample_model = Deep4Net(n_chans, n_classes,
                 input_time_length=input_time_length,
                 pool_time_length=2,
                 pool_time_stride=2,
                 final_conv_length='auto')
from braindecode.torch_ext.optimizers import AdamW
import torch.nn.functional as F
optimizer = AdamW(sample_model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
#optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)

sample_model.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1,)

In [ ]:
sample_model.fit(sample_set.X, sample_set.y, epochs=30, batch_size=64, scheduler='cosine',
         validation_data=(test_set.X, test_set.y),)

In [ ]:
sample_model.evaluate(train_set.X, train_set.y)

In [ ]:
# First test, towards inception distance: train on generated signals with deep4